In [1]:
import sqlite3
con = sqlite3.connect("movies.db")
cur = con.cursor()

### Country

In [17]:
from enum import Enum
class Country(Enum):
    BRITAIN = 'Britain'
    NEWZEALAND = 'NewZealand'
    USA = 'USA'
    BRAZIL = 'Brazil'

    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_countries (country TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO l_countries VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_countries')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_countries')
        sqlResult = cur.fetchall()
        return [Country(p[0]) for p in sqlResult]

In [3]:
Country.removeTable()
Country.createTable()

In [4]:
Country.getAll()

[<Country.BRITAIN: 'Britain'>,
 <Country.UGANDA: 'Uganda'>,
 <Country.SPAIN: 'Spain'>,
 <Country.BRAZIL: 'Brazil'>]

### Genre

In [5]:
class Genre(Enum):
    ADVENTURE = 'adventure'
    SCIFY = 'scify'
    ACTION = 'action'
    SUPERHERO = 'superhero'
    OTHERS = 'others'
    
    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Genre]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre text)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', Genre.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        return [Genre(p[0]) for p in sqlResult]

In [6]:
Genre.removeTable()
Genre.createTable()
Genre.getAll()

[<Genre.ADVENTURE: 'adventure'>,
 <Genre.SCIFY: 'scify'>,
 <Genre.ACTION: 'action'>,
 <Genre.SUPERHERO: 'superhero'>,
 <Genre.OTHERS: 'others'>]

In [7]:
def createGenreMapping():
    cur.execute('''CREATE TABLE IF NOT EXISTS MOVIE_GENRE(
                    movie text,
                    genre text,
                    PRIMARY KEY(movie, genre),
                    FOREIGN KEY(movie) REFERENCES movies(name),
                    FOREIGN KEY(genre) REFERENCES l_genres(genre))''')
    con.commit()

In [8]:
cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [9]:
createGenreMapping()

### Movie

In [11]:
class Movie:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name text PRIMARY KEY,
                        director text,
                        country text,
                        year integer,
                        rating REAL,
                        FOREIGN KEY(country) REFERENCES l_countries(country))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?, ?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, m.value) for m in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', (self.director, self.country, self.year, self.rating, self.name))
        con.commit()
        
    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS movies')
        con.commit()
        
    @classmethod
    def get(self, name):
        cur.execute(''' SELECT director, country, year, rating, GROUP_CONCAT(genre, ',') FROM movies
                    LEFT JOIN movie_genre ON movies.name = movie_genre.movie
                    WHERE movies.name=? GROUP BY movies.name''', (name,))
        res = cur.fetchone()
        if res:
            return Movie(name, res[0], res[1], res[2], res[3], getGenresFromString(res[4]))
        else:
            return None

    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)
    

def getGenresFromString(str):
    return {Genre(e) for e in str.split(',')}

In [13]:
Movie.removeTable()
Movie.createTable()

In [20]:
movie_1 = Movie('Lord of the rings 1', 'Peter Jackson', 'NewZeaLand', 2001, 8.9, {Genre.ADVENTURE})
movie_2 = Movie('Spider-Man', 'Sam Raimi', 'USA', 2002, 7.4, {Genre.SUPERHERO, Genre.ACTION})

movie_1.insert()
movie_2.insert()

In [21]:
print(Movie.get('Lord of the rings 1'))

(Lord of the rings 1, Peter Jackson, NewZeaLand, 2001, 8.9, {<Genre.ADVENTURE: 'adventure'>})


### Director

In [22]:
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INTEGER,
                        country TEXT)''')
        con.commit()

    @classmethod
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country, self.name))
        con.commit() 

    @classmethod
    def get(self, name):
        cur.execute('SELECT birth_year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        if res:
            return Director(name, res[0], res[1])
        else:
            return None

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, birth_year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country)

In [23]:
Director.removeTable()
Director.createTable()

In [24]:
d1 = Director('Peter Jackson',  1961, 'NewZealand')
d2 = Director('Sam Raimi', 1959, 'USA')

d1.insert()
d2.insert()

In [25]:
directorFromDb = Director.get('Peter Jackson')
print(directorFromDb)

(Peter Jackson, 1961, NewZealand)


In [26]:
class Movies:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name TEXT PRIMARY KEY,
                        director TEXT,
                        country TEXT,
                        year INTEGER,
                        rating REAL,
                        FOREIGN KEY(director) REFERENCES directors(name))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?,?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, genre.value) for genre in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', 
                    (self.director, self.country, self.year, self.rating, self.name))
        con.commit() 


    @classmethod
    def get(self):
        cur.execute('''
                    SELECT m.name, m.director, d.birth_year, d.country, m.country, m.year, m.rating, GROUP_CONCAT(g.genre, ',')
                    FROM movies m
                    LEFT JOIN directors d ON m.director = d.name
                    LEFT JOIN movie_genre g ON m.name = g.movie
                    GROUP BY m.name
                    ''')
        res = cur.fetchall()
        return res
    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)

In [28]:
Movie.removeTable()
Movie.createTable()

In [29]:
cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [30]:
createGenreMapping()

In [31]:
movie_1 = Movie('Lord of the rings 1', 'Peter Jackson', 'NewZeaLand', 2001, 8.9, {Genre.ADVENTURE})
movie_2 = Movie('Spider-Man', 'Sam Raimi', 'USA', 2002, 7.4, {Genre.SUPERHERO, Genre.ACTION})

movie_1.insert()
movie_2.insert()

In [55]:
print(movie_1.get("Lord of the rings 1"))

(Lord of the rings 1, Peter Jackson, NewZeaLand, 2001, 8.9, {<Genre.ADVENTURE: 'adventure'>})


In [60]:
movies = movie_2.get()
formatted_movies = []
for movie in movies:
    formatted_movies.append(str(movie))
print(formatted_movies)

TypeError: Movie.get() missing 1 required positional argument: 'name'

### erDiagram

erDiagram


    movie {
        TEXT name PK
        TEXT director
        TEXT country FK
        INTEGER year
        REAL rating

    }

    l_countries {
        TEXT country PK
    }

    directors {
        TEXT name PK
        INTEGER year
        TEXT country
    }

    l_genres {
        TEXT genre PK
    }

    movie_genres {
        TEXT movie FK
        TEXT genre FK
    }


    movie }|--|| directors : "made by"
    movie }|--|| l_countries : from
    movie }|--|{ movie_genres  : "is"
    l_genres||--|{ movie_genres : refers